# Compute DSM step by step

This notebook explains how to run step by step DSM computation with CARS, starting from the prepare step output folder.

## Notebook parameters

Those parameters need to be set before running the notebook.

In [ ]:
# Path to the cars folder
cars_home = "TODO"
# Path to the directory containing the content.json file of the prepare step output
content_dir = "TODO"
# ROI to process (roi_file = path to a vector file, raster file or roi_bbox=bounding box), as expected by cars command
# Use one or the other (roi_file will have precedence if not None)
roi_file = "TODO"  # Put roi_file=None to use roi_bbox
roi_bbox = ["xmin", "ymin", "xmax", "ymax"] # Use 4 floats value
# Path to output dir where to save figures and data
output_dir = "TODO"

## Imports

In [ ]:
### Trick to override cars version
import sys
sys.path = [cars_home] + sys.path
import os
import math
os.environ['OTB_APPLICATION_PATH'] = os.path.join(cars_home,'build','lib','otb','applications')+':'+os.environ['OTB_APPLICATION_PATH']
###
# Silent OTB info logs
os.environ['OTB_LOGGER_LEVEL']='WARNING'
import warnings
warnings.filterwarnings("ignore",category=UserWarning)
import xarray as xr
import numpy as np
%matplotlib inline
import matplotlib as mp
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LightSource
from matplotlib import pyplot as plt

from cars.conf import output_prepare
from cars.externals.matching.correlator_configuration import corr_conf
from cars.core import constants as cst
from cars.core import utils, projection, tiling, outputs
from cars.cars import parse_roi_file
from cars.steps import rasterization, points_cloud, triangulation
from cars.steps.epi_rectif import resampling
from cars.steps.matching import dense_matching
import pandora.marge

## Read data

First step is to read the content file from the prepare step, and to retrieve the disparity range. Note that disparity range can be changed here for experimentations.

In [ ]:
conf = output_prepare.read_preprocessing_content_file(os.path.join(content_dir,'content.json'))
disp_min = int(math.floor(conf['preprocessing']['output']['minimum_disparity']))
disp_max = int(math.ceil(conf['preprocessing']['output']['maximum_disparity']))

We also need to set up a configuration for pandora, the disparity estimation algorithm.

In [ ]:
corr_conf = corr_conf.configure_correlator()

## Compute the Region Of Interest to Process

First, we need to define the region of interest that will be processed by the notebook. For that, use `roi_file` ( a vector or a raster file) or `roi_bbox` (four float array).

In [ ]:
if roi_file is not None:
    bounds, stop_now = parse_roi_file(roi_file, stop_now=True)
else: 
    bounds = (roi_bbox, None)
print("Bounds: {}, EPSG={}".format(bounds[0], bounds[1]))

Now, we need to compute the corresponding epipolar region:

In [ ]:
epipolar_region = tiling.transform_terrain_region_to_epipolar(bounds[0], conf, bounds[1], disp_min, disp_max)
print("Corresponding epipolar region: {} (size: {} x {} pixels)".format(epipolar_region, epipolar_region[2]-epipolar_region[0], epipolar_region[3]-epipolar_region[1]))

## Stereo-rectify images

Before rectifying the images, we need to compute the margins needed by the disparity estimation algorithm:

In [ ]:
margins = pandora.marge.get_margins(disp_min, disp_max, corr_conf['pipeline'])
margins

Now we can call the images rectification function. It will return 3 datasets, respectively for the left image, right image and left color image.

In [ ]:
left_dataset, right_dataset, left_color_dataset = resampling.epipolar_rectify_images(conf, epipolar_region, margins)

Lets display the left and right images with their masks, and see if epipolar geometry is ok with an horizontal red line.

In [ ]:
fig_size = 15
clip_percent = 5
vmin_left = np.percentile(left_dataset[cst.EPI_IMAGE],clip_percent)
vmax_left = np.percentile(left_dataset[cst.EPI_IMAGE],100-clip_percent)
vmin_right = np.percentile(right_dataset[cst.EPI_IMAGE],clip_percent)
vmax_right = np.percentile(right_dataset[cst.EPI_IMAGE],100-clip_percent)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(fig_size, 1.05 * fig_size / 2), subplot_kw={'aspect': 1})
axes[0].set_title("Left image")
axes[0].imshow(left_dataset[cst.EPI_IMAGE], cmap="gray", interpolation='spline36', vmin=vmin_left, vmax=vmax_left)
axes[0].imshow(left_dataset[cst.EPI_MSK].where(left_dataset[cst.EPI_MSK] !=0), cmap='tab10', alpha=0.5)
axes[0].axhline(len(left_dataset[cst.ROW])/2., color='red')
axes[1].set_title("Right image")
axes[1].imshow(right_dataset[cst.EPI_IMAGE], cmap="gray", interpolation='spline36', vmin=vmin_right, vmax=vmax_right)
axes[1].imshow(right_dataset[cst.EPI_MSK].where(right_dataset[cst.EPI_MSK] !=0), cmap='tab10', alpha=0.5)
axes[1].axhline(len(right_dataset[cst.ROW])/2., color='red')
fig.tight_layout()
fig.savefig(os.path.join(output_dir,'epipolar_images.pdf'))

The next cell computes the anaglyph, which we will use later.

In [ ]:
left_dataset, right_dataset_align = xr.align(left_dataset,right_dataset)
anaglyph = np.stack((left_dataset[cst.EPI_IMAGE],right_dataset_align[cst.EPI_IMAGE], right_dataset_align[cst.EPI_IMAGE]),axis=-1)

In next cell we display the color image as well.

In [ ]:
left_color_dataset = left_color_dataset.where(left_color_dataset[cst.EPI_IMAGE] != 0)
color_min = left_color_dataset[cst.EPI_IMAGE].quantile(clip_percent/100., dim=[cst.ROW, cst.COL])
color_max = left_color_dataset[cst.EPI_IMAGE].quantile((100-clip_percent)/100., dim=[cst.ROW, cst.COL])
left_color_rescaled_dataset = (left_color_dataset[cst.EPI_IMAGE]-color_min)/(color_max-color_min)
red_id = 0
green_id = 0
blue_id = 0
if len(left_color_rescaled_dataset[cst.BAND]) > 2:
    green_id = 1
    blue_id = 2
red = left_color_rescaled_dataset.values[red_id,:,:]
green = left_color_rescaled_dataset.values[green_id,:,:]
blue = left_color_rescaled_dataset.values[blue_id,:,:]
rgb = np.stack((red,green,blue), axis=-1)
rgb = np.clip(rgb,0,1)
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(fig_size, 1.05 * fig_size / 2), subplot_kw={'aspect': 1})
axes.imshow(rgb, cmap="gray", interpolation='spline36')
fig.savefig(os.path.join(output_dir,'left_color_image.pdf'))

## Compute Disparity

Now that we have stereo-rectified images, we can compute the disparity:

In [ ]:
disp = dense_matching.compute_disparity(left_dataset, right_dataset, conf, corr_conf, disp_min, disp_max, verbose = True, use_sec_disp=False)

We crop out margins, since we do not need them anymore.

In [ ]:
left_dataset, tmp = xr.align(left_dataset,disp[cst.STEREO_REF])

Let's display left image along with estimated disparity map. Disparity map is overlayed with masked input pixels (gray), detected occlusions (red) and detected false matches (orange).

In [ ]:
vmin_anaglyph = np.array([vmin_left, vmin_right, vmin_right])
vmax_anaglyph = np.array([vmax_left, vmax_right, vmax_right])
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(fig_size, 1.05 * fig_size / 2), subplot_kw={'aspect': 1})
axes[0].set_title("Anaglyph")
im0 = axes[0].imshow(np.clip((anaglyph-vmin_anaglyph)/(vmax_anaglyph-vmin_anaglyph),0,1), cmap="gray", interpolation='spline36')
fig.colorbar(im0,  ax=axes[0], orientation='horizontal', fraction=0.1)
axes[1].set_title("Disparity map")
im1 = axes[1].imshow(disp[cst.STEREO_REF][cst.DISP_MAP], cmap="viridis", vmin=disp_min, vmax=disp_max)
axes[1].imshow(left_dataset[cst.EPI_MSK].where(left_dataset[cst.EPI_MSK] !=0), cmap='Set1',alpha=1, vmin=0, vmax=255)
# Will display in red
axes[1].imshow(disp[cst.STEREO_REF][cst.DISP_MSK_OCCLUSION].where(disp[cst.STEREO_REF][cst.DISP_MSK_OCCLUSION] ==0), cmap='Set1', alpha=1, vmin=0, vmax=255)
# Will display in yellow (+140), see color map Set1
axes[1].imshow(disp[cst.STEREO_REF][cst.DISP_MSK_FALSE_MATCH].where(disp[cst.STEREO_REF][cst.DISP_MSK_FALSE_MATCH] ==0)+140, cmap='Set1', alpha=1, vmin=0, vmax=255)
fig.colorbar(im1,  ax=axes[1], orientation='horizontal', fraction=0.1)
fig.tight_layout()
fig.savefig(os.path.join(output_dir,'disparity_map.pdf'))

## Triangulation

Now that we estimated the disparity, we can triangulate it to get WGS84 3D points:

In [ ]:
cloud = triangulation.triangulate(conf, disp[cst.STEREO_REF])

## Rasterization

We will rasterize in the UTM local projection. First thing is to determine the UTM zone to use. There is a convenient function for that, which we call on the first point of our points cloud.

In [ ]:
utm_zone = projection.get_utm_zone_as_epsg_code(cloud[cst.STEREO_REF][cst.X][0,0], cloud[cst.STEREO_REF][cst.Y][0,0])
print("UTM zone derived from point cloud: {}".format(utm_zone))

We will rasterize at 0.5 meters resolution. Small clusters of 3D points which are linked by a distance of 3 (less than 50 points) will be removed from the point cloud. A statistical filter is also applied on the points cloud to remove outliers by looking at the distribution of the mean distances between each point and its k neighbors. The filtered elements mask will be added to the cloud[cst.STEREO_REF] dataset. Feel free to change rasterization parameters.

Small components filtering parameters description:
* on_ground_margin: margin added to the on ground region to not filter points clusters that were incomplete because they were on the edges. Set to 0 here has there is no tiling.
* pts_connection_dist: distance to use to consider that two points are connected
* nb_pts_threshold: point clusters that have less than this number of points will be filtered
* dist_between_clusters: distance to use to consider that two points clusters are far from each other or not. If a small points cluster is near to another one, it won't be filtered. (None = deactivated)
* construct_removed_elt_msk: if set to True, the removed points mask will be added to the cloud datasets in input of the simple_rasterization_dataset)
* mask_value: value to use to identify the removed points in the mask

Statistical filtering parameters description:
* k: number of neighbors
* stdev_factor: factor to apply in the distance threshold computation
* construct_removed_elt_msk: if set to True, the removed points mask will be added to the cloud datasets in input of the simple_rasterization_dataset
* mask_value: value to use to identify the removed points in the mask


In [ ]:
resolution = 0.5 # meters
radius = 1 # pixels
sigma = 0.3 # pixels

# filtering params
on_ground_margin = 0
pts_connection_dist = 3
nb_pts_threshold = 50
dist_between_clusters = None #None = deactivated
construct_removed_elt_msk = True
mask_value = 255
small_cpn_filter_params = points_cloud.SmallComponentsFilterParams(on_ground_margin,
                                                                pts_connection_dist,
                                                                nb_pts_threshold,
                                                                dist_between_clusters,
                                                                construct_removed_elt_msk,
                                                                mask_value)

k = 50
std_dev_factor = 5
construct_removed_elt_msk = True
mask_value = 255
statistical_filter_params = points_cloud.StatisticalFilterParams(k, std_dev_factor, construct_removed_elt_msk, mask_value)

# project in the correct epsg code referential (ecef if filters are activated, utm_zone otherwise)
if small_cpn_filter_params or statistical_filter_params:
    projection.points_cloud_conversion_dataset(cloud[cst.STEREO_REF], 4978)
else:
    projection.points_cloud_conversion_dataset(cloud[cst.STEREO_REF], utm_zone)

# rasterization
if small_cpn_filter_params is not None or statistical_filter_params is not None:
    raster, filtered_points = rasterization.simple_rasterization_dataset([cloud[cst.STEREO_REF]], resolution, utm_zone, [left_color_dataset], radius=radius, sigma=sigma,
                                                                         small_cpn_filter_params=small_cpn_filter_params, statistical_filter_params=statistical_filter_params,
                                                                         dump_filter_cloud=True)
else:
    raster = rasterization.simple_rasterization_dataset([cloud[cst.STEREO_REF]], resolution, utm_zone, [left_color_dataset], radius=radius, sigma=sigma)

Lets display the rasterized DSM and color image side by side.

In [ ]:
ls = LightSource(azdeg=315, altdeg=70)
hmin = raster[cst.RASTER_HGT].min()
hmax = raster[cst.RASTER_HGT].max()
red = (raster[cst.RASTER_COLOR_IMG].values[red_id,:,:]-color_min.values[red_id])/(color_max.values[red_id]-color_min.values[red_id])
green = (raster[cst.RASTER_COLOR_IMG].values[green_id,:,:]-color_min.values[green_id])/(color_max.values[green_id]-color_min.values[green_id])
blue = (raster[cst.RASTER_COLOR_IMG].values[blue_id,:,:]-color_min.values[blue_id])/(color_max.values[blue_id]-color_min.values[blue_id])
rgb = np.stack((red,green,blue), axis=-1)
rgb = np.clip(rgb,0,1)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(fig_size, 1.05 * fig_size / 2))
axes[0].set_title("DSM (meters)")
im0 = axes[0].imshow(ls.shade(raster[cst.RASTER_HGT].values, cmap=plt.cm.terrain, blend_mode='soft', vert_exag=10, dx=resolution, dy=resolution, vmin=hmin, vmax=hmax))
axes[0].grid(True)
axes[1].set_title("Ortho")
axes[1].imshow(rgb, interpolation='nearest')
axes[1].grid(True)
fig.tight_layout()
fig.savefig(os.path.join(output_dir,'dsm_and_color.pdf'))

We can also display usefull statistics, such as the standard deviation or number of points in each cell.

In [ ]:
clip_percent = 2
std_min = np.nanpercentile(raster[cst.RASTER_HGT_STD_DEV],clip_percent)
std_max = np.nanpercentile(raster[cst.RASTER_HGT_STD_DEV],100-clip_percent)
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(fig_size, 1.05 * fig_size / 2))
axes[0].set_title("Cells standard deviation (meters)")
im0 = axes[0].imshow(raster[cst.RASTER_HGT_STD_DEV], cmap="YlOrRd", interpolation='nearest', vmin=std_min, vmax=std_max)
fig.colorbar(im0,  ax=axes[0], orientation='vertical', fraction=0.1)
axes[0].grid(True)
axes[1].set_title("Cells number of points")
im1 = axes[1].imshow(raster[cst.RASTER_NB_PTS].where(raster[cst.RASTER_NB_PTS]!=0), cmap="YlOrRd", interpolation='nearest')
fig.colorbar(im1,  ax=axes[1], orientation='vertical', fraction=0.1)
axes[1].grid(True)
fig.tight_layout()
fig.savefig(os.path.join(output_dir,'dsm_stdev_and_nb_points.pdf'))

## Visualize point cloud

Caution: this part may be very long to execute in the notebook. In this part we will display the colorized points cloud in 3D. First step is rescale colors and filter nan values in color and cloud.



In [ ]:
cloud_filtered = cloud[cst.STEREO_REF].where(cloud[cst.STEREO_REF][cst.POINTS_CLOUD_CORR_MSK] == 255)
color_filtered = left_color_dataset.where(cloud[cst.STEREO_REF][cst.POINTS_CLOUD_CORR_MSK] == 255)
color_filtered = color_filtered.where(cloud[cst.STEREO_REF][cst.POINTS_CLOUD_CORR_MSK] == 255)
color_filtered = (color_filtered-color_min)/(color_max-color_min)
red = color_filtered[cst.EPI_IMAGE].values[red_id,:,:]
red = red[~np.isnan(red)]
green = color_filtered[cst.EPI_IMAGE].values[green_id,:,:]
green = green[~np.isnan(green)]
blue = color_filtered[cst.EPI_IMAGE].values[blue_id,:,:]
blue = blue[~np.isnan(blue)]
rgb = np.stack((red,green,blue), axis=-1)
rgb = np.clip(rgb,0,1)

Next, we compute the correct scaling for all axis (autoscaling is not good, since the z axis probably spans a lot less meters than the X and Y axes)

In [ ]:
centerx = np.mean(cloud_filtered[cst.X])
centery = np.mean(cloud_filtered[cst.Y])
centerz = np.mean(cloud_filtered[cst.Z])
widthx = max(abs(np.min(cloud_filtered[cst.X]-centerx)),abs(np.max(cloud_filtered[cst.X]-centerx)))
widthy = max(abs(np.min(cloud_filtered[cst.Y]-centery)),abs(np.max(cloud_filtered[cst.Y]-centery)))
widthz = max(abs(np.min(cloud_filtered[cst.Z]-centerz)),abs(np.max(cloud_filtered[cst.Z]-centerz)))
width = max(widthx, widthy, widthz)

Finally we can draw a 3D colorized scatter plot of the scene.

In [ ]:
fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(cloud_filtered[cst.X], cloud_filtered[cst.Y], cloud_filtered[cst.Z], c=rgb , marker='o', s=0.01)
ax.set_xlim(centerx-width, centerx+width)
ax.set_ylim(centery-width, centery+width)
ax.set_zlim(centerz-width, centerz+width)
fig.savefig(os.path.join(output_dir,'3d_view.pdf'))

## Saving data

Saving left, right and left color datasets:

In [ ]:
left_dataset.to_netcdf(os.path.join(output_dir, "left_dataset.nc"))
right_dataset.to_netcdf(os.path.join(output_dir, "right_dataset.nc"))
left_color_dataset.to_netcdf(os.path.join(output_dir, "left_color_dataset.nc"))

Saving dispariy:

In [ ]:
disp[cst.STEREO_REF].to_netcdf(os.path.join(output_dir, "disparity.nc"))

Saving triangulated cloud:

In [ ]:
cloud[cst.STEREO_REF].to_netcdf(os.path.join(output_dir, "cloud.nc"))

Saving DSM:

In [ ]:
raster.to_netcdf(os.path.join(output_dir, "dsm.nc"))

Save cloud in ply file format:

In [ ]:
outputs.write_ply(os.path.join(output_dir,"points.ply"),cloud[cst.STEREO_REF])
if small_cpn_filter_params is not None or statistical_filter_params is not None:
    outputs.write_ply(os.path.join(output_dir,"filtered_points.ply"),filtered_points)